# Validated Ensemble with Strict Overlap Checking

Scan all snapshots for better per-N configurations, validate strictly for overlaps, and fall back to baseline for any failing configurations.

In [ ]:
import pandas as pd
import numpy as np
import math
from numba import njit
import json
import os
from shapely.geometry import Polygon
from shapely import affinity
from collections import defaultdict

# Tree vertices
TX = np.array([0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125])
TY = np.array([0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5])

In [ ]:
@njit
def score_group(xs, ys, degs, tx, ty):
    """Calculate score for a single N configuration"""
    n = xs.size
    V = tx.size
    mnx = mny = 1e300
    mxx = mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r)
        s = math.sin(r)
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xs[i]
            Y = s * tx[j] + c * ty[j] + ys[i]
            mnx = min(mnx, X)
            mxx = max(mxx, X)
            mny = min(mny, Y)
            mxy = max(mxy, Y)
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

In [ ]:
def make_tree_polygon(x, y, deg):
    """Create a Shapely polygon for a tree at given position and rotation"""
    p = Polygon(zip(TX, TY))
    p = affinity.rotate(p, deg, origin=(0,0))
    p = affinity.translate(p, x, y)
    return p

def check_overlaps_strict(xs, ys, degs, buffer_dist=1e-6):
    """Check for overlaps with strict tolerance (buffer to catch near-overlaps)"""
    n = len(xs)
    if n <= 1:
        return True, []
    
    # Create polygons with small buffer to catch near-overlaps
    polygons = []
    for i in range(n):
        p = make_tree_polygon(xs[i], ys[i], degs[i])
        # Apply negative buffer to be stricter (polygons must not touch)
        polygons.append(p)
    
    overlaps = []
    for i in range(n):
        for j in range(i+1, n):
            # Check if polygons overlap (not just touch)
            if polygons[i].intersects(polygons[j]):
                # Calculate minimum distance
                dist = polygons[i].distance(polygons[j])
                if dist < buffer_dist:
                    # Check if they actually overlap (not just touch)
                    intersection = polygons[i].intersection(polygons[j])
                    if intersection.area > 1e-12:  # Non-trivial overlap
                        overlaps.append((i, j, intersection.area))
    
    return len(overlaps) == 0, overlaps

In [ ]:
def parse_submission(filepath):
    """Parse submission CSV and return dict of N -> (xs, ys, degs)"""
    df = pd.read_csv(filepath)
    
    # Parse values (remove 's' prefix)
    df['x_val'] = df['x'].str.replace('s', '').astype(float)
    df['y_val'] = df['y'].str.replace('s', '').astype(float)
    df['deg_val'] = df['deg'].str.replace('s', '').astype(float)
    
    # Extract N from id (format: NNN_idx)
    df['N'] = df['id'].str.split('_').str[0].astype(int)
    
    configs = {}
    for n, group in df.groupby('N'):
        xs = group['x_val'].values
        ys = group['y_val'].values
        degs = group['deg_val'].values
        configs[n] = (xs, ys, degs)
    
    return configs

In [ ]:
def calculate_total_score(configs, tx, ty):
    """Calculate total score across all N values"""
    total = 0.0
    scores_by_n = {}
    for n in range(1, 201):
        if n in configs:
            xs, ys, degs = configs[n]
            score = score_group(xs, ys, degs, tx, ty)
            scores_by_n[n] = score
            total += score
    return total, scores_by_n

In [ ]:
# Load baseline (known valid)
baseline_path = '/home/code/preoptimized/submission.csv'
baseline_configs = parse_submission(baseline_path)
baseline_total, baseline_scores = calculate_total_score(baseline_configs, TX, TY)
print(f"Baseline score: {baseline_total:.6f}")

# Verify baseline passes strict overlap check for N=40 (the failing group)
xs, ys, degs = baseline_configs[40]
valid, overlaps = check_overlaps_strict(xs, ys, degs)
print(f"Baseline N=40 valid: {valid}, overlaps: {len(overlaps)}")

In [ ]:
# Scan all snapshot directories for submission files
snapshot_base = '/home/nonroot/snapshots/santa-2025/'
snapshot_dirs = os.listdir(snapshot_base)

all_sources = {}
all_sources['baseline'] = baseline_configs

print(f"Scanning {len(snapshot_dirs)} snapshot directories...")

for snap_dir in sorted(snapshot_dirs):
    snap_path = os.path.join(snapshot_base, snap_dir)
    
    # Check for submission.csv in various locations
    possible_paths = [
        os.path.join(snap_path, 'submission', 'submission.csv'),
        os.path.join(snap_path, 'code', 'submission.csv'),
    ]
    
    for path in possible_paths:
        if os.path.exists(path):
            try:
                configs = parse_submission(path)
                if len(configs) == 200:  # Valid submission has 200 N values
                    total, _ = calculate_total_score(configs, TX, TY)
                    all_sources[f'snap_{snap_dir}'] = configs
            except Exception as e:
                pass
            break

print(f"Loaded {len(all_sources)} sources")

In [ ]:
# Also load preoptimized files
preopt_files = [
    '/home/code/preoptimized/smartmanoj_submission.csv',
    '/home/code/preoptimized/saspav_best.csv',
]

for path in preopt_files:
    if os.path.exists(path):
        try:
            configs = parse_submission(path)
            if len(configs) == 200:
                name = os.path.basename(path).replace('.csv', '')
                all_sources[name] = configs
                total, _ = calculate_total_score(configs, TX, TY)
                print(f"Loaded {name}: {total:.6f}")
        except Exception as e:
            print(f"Error loading {path}: {e}")

In [ ]:
# For each N, find the best configuration that passes strict overlap validation
print("\nFinding best valid configuration for each N...")

best_configs = {}
best_scores_by_n = {}
source_used = {}
validation_failures = []

for n in range(1, 201):
    best_score = float('inf')
    best_config = None
    best_source = None
    
    # Collect all candidates for this N
    candidates = []
    for source_name, configs in all_sources.items():
        if n in configs:
            xs, ys, degs = configs[n]
            score = score_group(xs, ys, degs, TX, TY)
            candidates.append((score, source_name, (xs, ys, degs)))
    
    # Sort by score (best first)
    candidates.sort(key=lambda x: x[0])
    
    # Try each candidate until we find one that passes validation
    for score, source_name, config in candidates:
        xs, ys, degs = config
        valid, overlaps = check_overlaps_strict(xs, ys, degs)
        
        if valid:
            best_score = score
            best_config = config
            best_source = source_name
            break
        else:
            if source_name != 'baseline':
                validation_failures.append((n, source_name, score, len(overlaps)))
    
    # If no valid config found, use baseline (should always be valid)
    if best_config is None:
        xs, ys, degs = baseline_configs[n]
        best_score = baseline_scores[n]
        best_config = (xs, ys, degs)
        best_source = 'baseline_fallback'
    
    best_configs[n] = best_config
    best_scores_by_n[n] = best_score
    source_used[n] = best_source

print(f"\nValidation failures (better score but overlapping): {len(validation_failures)}")
for n, source, score, num_overlaps in validation_failures[:10]:
    print(f"  N={n}: {source} score={score:.6f} overlaps={num_overlaps}")

In [ ]:
# Calculate final score
final_score = sum(best_scores_by_n.values())
print(f"\nFinal ensemble score: {final_score:.6f}")
print(f"Baseline score: {baseline_total:.6f}")
print(f"Improvement: {baseline_total - final_score:.6f}")

# Source breakdown
source_counts = defaultdict(int)
for n, source in source_used.items():
    source_counts[source] += 1

print(f"\nSource breakdown:")
for source, count in sorted(source_counts.items(), key=lambda x: -x[1])[:10]:
    print(f"  {source}: {count} N values")

In [ ]:
# Show which N values improved
print("\nN values with improvement over baseline:")
improvements = []
for n in range(1, 201):
    if best_scores_by_n[n] < baseline_scores[n] - 1e-9:
        imp = baseline_scores[n] - best_scores_by_n[n]
        improvements.append((n, imp, source_used[n]))

improvements.sort(key=lambda x: -x[1])
for n, imp, source in improvements[:20]:
    print(f"  N={n}: +{imp:.6f} from {source}")

print(f"\nTotal improvements: {len(improvements)} N values")
print(f"Total improvement: {sum(x[1] for x in improvements):.6f}")

In [ ]:
# Generate submission CSV
def generate_submission(configs, output_path):
    rows = []
    for n in range(1, 201):
        xs, ys, degs = configs[n]
        for i in range(len(xs)):
            row = {
                'id': f'{n:03d}_{i}',
                'x': f's{xs[i]}',
                'y': f's{ys[i]}',
                'deg': f's{degs[i]}'
            }
            rows.append(row)
    df = pd.DataFrame(rows)
    df.to_csv(output_path, index=False)
    return df

# Save submission
os.makedirs('/home/submission', exist_ok=True)
submission_df = generate_submission(best_configs, '/home/submission/submission.csv')
print(f"Saved submission to /home/submission/submission.csv")
print(f"Total rows: {len(submission_df)}")

In [ ]:
# Verify the saved submission
verify_configs = parse_submission('/home/submission/submission.csv')
verify_total, verify_scores = calculate_total_score(verify_configs, TX, TY)
print(f"\nVerification of saved submission: {verify_total:.6f}")
print(f"Expected: {final_score:.6f}")
print(f"Match: {abs(verify_total - final_score) < 1e-6}")

# Double-check N=40 passes validation
xs, ys, degs = verify_configs[40]
valid, overlaps = check_overlaps_strict(xs, ys, degs)
print(f"\nN=40 validation: valid={valid}, overlaps={len(overlaps)}")

In [ ]:
# Save metrics
metrics = {
    'cv_score': verify_total,
    'baseline_score': baseline_total,
    'improvement': baseline_total - verify_total,
    'num_improvements': len(improvements),
    'validation_failures': len(validation_failures)
}

with open('/home/code/experiments/003_bbox3_optimization/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"\nSaved metrics to experiments/003_bbox3_optimization/metrics.json")
print(f"CV Score: {verify_total:.6f}")